## Mathematical exercises: Pinhole camera
In these exercises we will assume a $modern$ camera with completely square pixels. What are the skew parameters then?

### Exercise 2.1
Reuse the box3d function from last week. Assume that $f$ = 600, $α$ = 1, $β$ = 0, and ${\delta}_x$ = ${\delta}_y$ = 400.
Given a traditional camera, what is the resolution in pixels?
>In a traditional camera, the principal point is exactly in the middle of the sensor. So, for this
camera the sensor has 2 × 400 = 800 pixels along each dimension i.e. a resolution of 800 × 800
pixels.

Also assume $R$ = $I$, and $t = \begin{bmatrix}
0 \\ 0.2 \\ 1.5
\end{bmatrix}$
. Use projectpoints from last week, to project the
points.

Are all the points are captured by the image sensor?
Where does the corner $P_1$ = [−0.5,−0.5,−0.5] project to?
>The projection matrix reads $p = \begin{bmatrix}
600 & 0 & 400 & 600 \\ 0 & 600 & 400 & 720  \\ 0 & 0 & 1 & 1.5
\end{bmatrix}$, Some points have an $y$ value greater than 800, and are not visible in the image, as they are outside
the image sensor. $P_1$ projects to $[100,400]^T$

### Exercise 2.2